**Fibonacci Sequence Example**

Here, we're going to implement our own Fibonacci sequence type and we want to support slicing.

We'll make our sequence type bounded (i.e. we'll have to specify the size of the sequence). But we are not going to pre-generate the entire sequence of Fibonacci numbers, we'll only generate the ones that are being requested as needed.

So, when we create our object, we'll have to specify the maximum Fibonacci number that will be possible for this sequence (but we won't generate that value or any other Fibonacci value during the creation of the object). See below.

```
f = Fib(10)    # Creating our Fibonacci sequence that can generate up to 10 values.
f[3:5]         # It can handle slices. In this case, return a list of the element 3 and 4 of the Fibonacci sequence (i.e. 4th and 5th).
f[::-1]        # Return a list of all Fibonacci values in reverse order. In this case, the 10th value and below. 
```

Here are the steps:


1. First we'll need to handle requesting a single index e.g. `f[3]` for the 4th Fibonacci value. To do this, we just need `__getitem__` which will allow the `[]` notation. 


2. To actually calculate the nth Fibonacci number, we'll use memoisation as well (see lecture on decorators and memoisation if you need to refresh your memory on that).

3. To be able to use `list(f)`, we need to raise an `IndexError` exception when the index is out of bounds as this is one of the two essential criteria for iterating through a sequence. 

4. We also need to remap negative indices (for example `-1` should correspond to the last element of the sequence, and so on). This is done with:     

```
if s < 0:
    s = self._n + s
```

5. Now let's handle slice requests. If you recall from Section 05 - Slicing, you might think it to be difficult to implement all those rules. But remember, the slice method `indices(<length of sequence>)` returns a tuple of 3 values which correspond to the equivalent start, stop and step values. These can be converted into a range object which we can iterate through, e.g. using a list comprehension or a 'for' loop, computing the corresponding Fibonacci value at each step. We can go further by generating the full list to show us which indices are going to be asked for.

For example:
   
```
   my_slice = slice(2, -7, -2)          # When applied to a sequence s, it'd be equivalent to s[2:-7:-2]. Hard to see which indices we are requiring.

   list(range(*my_slice.indices(6)))    # If we pass in the length of our sequence (6) then it outputs that we're asking for...
   
   > [2, 0]                             # ...index 2 and index 0. So find _fib(2) and then _fib(0), but we've not implemented this extra detail below.
 ```
   
We're done.

In [27]:
from functools import lru_cache

class Fib:
    def __init__(self, n):
        self._n = n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            # single item requested
            if s < 0:
                s = self._n + s
            if s < 0 or s > self._n - 1:
                raise IndexError
            return self._fib(s)                                 # this is basically our 'else' statement.
            
        else:
            # slice being requested
            idx = s.indices(self._n)                            # the argument required for s.indices is the length of the sequence which is self._n
            rng = range(*idx)                                   # This will create a range sequence object e.g. range(0, 10, 2), corresponding to the indices
            return [self._fib(n) for n in rng]                  # in the Fibonacci sequence that we want.
            
    @staticmethod
    @lru_cache(2**32)
    def _fib(n):
        if n < 2:
            return 1
        else:
            return Fib._fib(n-1) + Fib._fib(n-2)

In [28]:
f = Fib(10)

print(f[0])
print(f[9])
print(list(f))
print(list(item for item in f))
print(f[2:9:2])

1
55
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
[2, 5, 13, 34]
